In [2]:
import pandas as pd
import numpy as np
pd.options.display.max_colwidth = 100
from datetime import datetime

In [2]:
df=pd.read_csv("records.csv")

In [4]:
df.head()

,Fighter,URL,Nationality,Nickname,Birthday,Height ft,Height cm,Opponent,Result,Method,Round,Time,Event,Date,Referee
0,Marcin Tybura,https://www.sherdog.com/fighter/Marcin-Tybura-86928,Poland,Tybur,"Nov 9, 1985","6'3""",190.5 cm,Tai Tuivasa,win,Technical Submission (Rear-Naked Choke),1,4:08,UFC Fight Night 239 - Tuivasa vs. Tybura,Mar / 16 / 2024,Herb Dean
1,Marcin Tybura,https://www.sherdog.com/fighter/Marcin-Tybura-86928,Poland,Tybur,"Nov 9, 1985","6'3""",190.5 cm,Tom Aspinall,loss,TKO (Elbow and Punches),1,1:13,UFC Fight Night 224 - Aspinall vs. Tybura,Jul / 22 / 2023,Marc Goddard
2,Marcin Tybura,https://www.sherdog.com/fighter/Marcin-Tybura-86928,Poland,Tybur,"Nov 9, 1985","6'3""",190.5 cm,Blagoy Ivanov,win,Decision (Unanimous),3,5:00,UFC Fight Night 218 - Lewis vs. Spivak,Feb / 04 / 2023,Mark Smith
3,Marcin Tybura,https://www.sherdog.com/fighter/Marcin-Tybura-86928,Poland,Tybur,"Nov 9, 1985","6'3""",190.5 cm,Alexander Romanov,win,Decision (Majority),3,5:00,UFC 278 - Usman vs. Edwards 2,Aug / 20 / 2022,Jason Herzog
4,Marcin Tybura,https://www.sherdog.com/fighter/Marcin-Tybura-86928,Poland,Tybur,"Nov 9, 1985","6'3""",190.5 cm,Alexander Volkov,loss,Decision (Unanimous),3,5:00,UFC 267 - Blachowicz vs. Teixeira,Oct / 30 / 2021,Marc Goddard


We have to check that there is no URL leading to two fighters, it would mean that we scraped data from two different fighters from one URL

In [3]:
filtered_df = df.groupby('URL').filter(lambda x: len(x['Fighter'].unique()) > 1)
print(filtered_df[['Fighter', "URL"]])

Empty DataFrame
Columns: [Fighter, URL]
Index: []


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54860 entries, 0 to 54859
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Fighter      54860 non-null  object
 1   URL          54860 non-null  object
 2   Nationality  54845 non-null  object
 3   Nickname     41907 non-null  object
 4   Birthday     54073 non-null  object
 5   Height ft    54744 non-null  object
 6   Height cm    54744 non-null  object
 7   Opponent     54860 non-null  object
 8   Result       54860 non-null  object
 9   Method       54764 non-null  object
 10  Round        54860 non-null  int64 
 11  Time         53665 non-null  object
 12  Event        54860 non-null  object
 13  Date         54860 non-null  object
 14  Referee      35550 non-null  object
dtypes: int64(1), object(14)
memory usage: 6.3+ MB


In [5]:
df.columns

Index(['Fighter', 'URL', 'Nationality', 'Nickname', 'Birthday', 'Height ft',
       'Height cm', 'Opponent', 'Result', 'Method', 'Round', 'Time', 'Event',
       'Date', 'Referee'],
      dtype='object')

### Fitting Methods into 4 possible categories - Decision, KO/TKO, Submission and No contest

Below we can see that there is too many Method categories

In [6]:
df["Method"].value_counts()

Method
Decision (Unanimous)                             13165
TKO (Punches)                                     8885
Submission (Rear-Naked Choke)                     5098
Decision (Split)                                  3086
Submission (Armbar)                               2613
                                                 ...  
No Contest (Llamas Injured by Illegal Strike)        1
Submission (Knee and Punches)                        1
No Contest(Accidental Elbow to Back of Head)         1
DQ (Intentional Headbutt)                            1
Submission (Leg Kicks)                               1
Name: count, Length: 733, dtype: int64

First we need to extract the description and store it into new column.

In [7]:
df

,Fighter,URL,Nationality,Nickname,Birthday,Height ft,Height cm,Opponent,Result,Method,Round,Time,Event,Date,Referee
0,Marcin Tybura,https://www.sherdog.com/fighter/Marcin-Tybura-86928,Poland,Tybur,"Nov 9, 1985","6'3""",190.5 cm,Tai Tuivasa,win,Technical Submission (Rear-Naked Choke),1,4:08,UFC Fight Night 239 - Tuivasa vs. Tybura,Mar / 16 / 2024,Herb Dean
1,Marcin Tybura,https://www.sherdog.com/fighter/Marcin-Tybura-86928,Poland,Tybur,"Nov 9, 1985","6'3""",190.5 cm,Tom Aspinall,loss,TKO (Elbow and Punches),1,1:13,UFC Fight Night 224 - Aspinall vs. Tybura,Jul / 22 / 2023,Marc Goddard
2,Marcin Tybura,https://www.sherdog.com/fighter/Marcin-Tybura-86928,Poland,Tybur,"Nov 9, 1985","6'3""",190.5 cm,Blagoy Ivanov,win,Decision (Unanimous),3,5:00,UFC Fight Night 218 - Lewis vs. Spivak,Feb / 04 / 2023,Mark Smith
3,Marcin Tybura,https://www.sherdog.com/fighter/Marcin-Tybura-86928,Poland,Tybur,"Nov 9, 1985","6'3""",190.5 cm,Alexander Romanov,win,Decision (Majority),3,5:00,UFC 278 - Usman vs. Edwards 2,Aug / 20 / 2022,Jason Herzog
4,Marcin Tybura,https://www.sherdog.com/fighter/Marcin-Tybura-86928,Poland,Tybur,"Nov 9, 1985","6'3""",190.5 cm,Alexander Volkov,loss,Decision (Unanimous),3,5:00,UFC 267 - Blachowicz vs. Teixeira,Oct / 30 / 2021,Marc Goddard
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54855,Razak Al-Hassan,https://www.sherdog.com/fighter/Razak-AlHassan-23326,United States,Razor,"May 14, 1982","6'3""",190.5 cm,Todd Monaghan,win,TKO (Doctor Stoppage),2,5:00,Mainstream MMA 9 - New Era,Apr / 05 / 2008,NaN
54856,Razak Al-Hassan,https://www.sherdog.com/fighter/Razak-AlHassan-23326,United States,Razor,"May 14, 1982","6'3""",190.5 cm,Eric Daughetee,win,Decision (Unanimous),3,5:00,Mainstream MMA 6 - Evolution,Jul / 14 / 2007,NaN
54857,Razak Al-Hassan,https://www.sherdog.com/fighter/Razak-AlHassan-23326,United States,Razor,"May 14, 1982","6'3""",190.5 cm,Demetrius Worlds,win,Submission (Rear-Naked Choke),3,1:01,TFC - Title Fighting Championships,Jun / 30 / 2007,NaN
54858,Razak Al-Hassan,https://www.sherdog.com/fighter/Razak-AlHassan-23326,United States,Razor,"May 14, 1982","6'3""",190.5 cm,Trevor Michaelis,win,Submission (Rear-Naked Choke),1,2:01,ECF - March of Champions,Mar / 24 / 2007,NaN


In [8]:
df['Description'] = df['Method'].str.extract(r'\((.*?)\)', expand=False)
df["Method"].replace(r'\((.*?)\)', "", regex=True, inplace=True)

We strip the columns from whitespaces.

In [9]:
df["Method"]=df["Method"].str.strip()
df["Description"]=df["Description"].str.strip()

In [10]:
print(df["Method"].value_counts())

Method
Decision                                 17135
Submission                               15501
TKO                                      14323
KO                                        5693
Technical Submission                       710
Draw                                       674
No Contest                                 409
Disqualification                           141
DQ                                          64
Technical Decision                          54
NC                                          43
Technical Draw                               8
ND                                           2
No Decision - Overturned by CSAC             1
No Contest - Collard Failed Drug Test        1
K.O                                          1
DG                                           1
Submission (Rear-Naked Choke                 1
Techinal Submission                          1
No Decision                                  1
Name: count, dtype: int64


There is way less categories now, but there are still some categories that can be merged to follow common practices.
1. "KO" and "TKO" can be merched together into "KO/TKO", "K.O" is a typo and also belong in this category
2. "Submission" and "Technical submission" can be merged into "Submission", "Techinal Submission" and "Submission (Rear-Naked Choke" is a typo and also belong in this category
3. "Draw" shouldnt be in "Method" column, it is a "Result" and all draws are "Decision"
4. "No Contest", "NC", "No Decision - Overturned by CSAC", "No Contest - Collard Failed Drug Test", "No Decision" and "ND" can be all merged as "No Contest", we will later get rid of them anyway as they mean that the match doesnt have any result
5. "Technical Decision" can be merged with "Decision" as it is only a different type of decision
6. "Disqualification" and "DQ" are commonly counted as "KO/TKO" so we can merge them as well, "DG" is a typo and also belong in this category
7. We will count "Technical Draw" as "Decision" as was stated in point 3.


In [11]:
# 1
df.loc[df["Method"] == "TKO", "Method"] = "KO/TKO"
df.loc[df["Method"] == "KO", "Method"] = "KO/TKO"
df.loc[df["Method"] == "K.O", "Method"] = "KO/TKO"
# 2
df.loc[df["Method"] == "Technical Submission", "Method"] = "Submission"
df.loc[df["Method"] == "Techinal Submission", "Method"] = "Submission"
df.loc[df["Method"] == "Submission (Rear-Naked Choke", "Method"] = "Submission"
# 3
df.loc[df["Method"] == "Draw", "Method"] = "Decision"
# 4
df.loc[df["Method"] == "NC", "Method"] = "No Contest"
df.loc[df["Method"] == "No Decision - Overturned by CSAC", "Method"] = "No Contest"
df.loc[df["Method"] == "No Decision", "Method"] = "No Contest"
df.loc[df["Method"] == "ND", "Method"] = "No Contest"
df.loc[df["Method"] == "No Contest - Collard Failed Drug Test", "Method"] = "No Contest"
# 5
df.loc[df["Method"] == "Technical Decision", "Method"] = "Decision"
# 6
df.loc[df["Method"] == "Disqualification", "Method"] = "KO/TKO"
df.loc[df["Method"] == "DQ", "Method"] = "KO/TKO"
df.loc[df["Method"] == "DG", "Method"] = "KO/TKO"
# 7
df.loc[df["Method"] == "Technical Draw", "Method"] = "Decision"

In [12]:
print(df["Method"].value_counts())

Method
KO/TKO        20223
Decision      17871
Submission    16213
No Contest      457
Name: count, dtype: int64


We have successfully fitted the methods into 4 categories 

### We can check other columns for more insight

In [13]:
print(df["Result"].value_counts())

Result
win     37299
loss    16420
draw      684
NC        457
Name: count, dtype: int64


we can see that results correspond with methods in the NC - No Contest count

In [14]:
print(df["Fighter"].value_counts())
print(df["URL"].value_counts())

Fighter
Travis Fulton       320
Dan Severn          127
Jeremy Horn         120
Ikuhisa Minowa      116
Yuki Kondo          115
                   ... 
Dave Berry            1
Scott Fiedler         1
Thomas Ramirez        1
Francesco Maturi      1
John Campatella       1
Name: count, Length: 2479, dtype: int64
URL
https://www.sherdog.com/fighter/Travis-Fulton-80        320
https://www.sherdog.com/fighter/Dan-Severn-52           127
https://www.sherdog.com/fighter/Jeremy-Horn-202         120
https://www.sherdog.com/fighter/Ikuhisa-Minowa-250      116
https://www.sherdog.com/fighter/Yuki-Kondo-263          115
                                                       ... 
https://www.sherdog.com/fighter/Dave-Berry-143            1
https://www.sherdog.com/fighter/Scott-Fiedler-138         1
https://www.sherdog.com/fighter/Thomas-Ramirez-125        1
https://www.sherdog.com/fighter/Francesco-Maturi-117      1
https://www.sherdog.com/fighter/John-Campetella-140       1
Name: count, Length: 247

We can see that "Fighter" and "URL" value counts have the same lenght, which is good, because both columns are unique identificators.

We are finished so we can save the table.

In [15]:
df.to_csv("records_clean.csv", index=False)

# Connecting two datasets - "records_clean.csv" and "bouts_new.csv"

Now we will use this table to calculate fighters records at the time of their fights in the ufc and connect them to the bouts dataset with additional info about fighters.

In records we will focus on:
1. wins by submission,
2. losses by submission,
3. wins by ko/tko,
4. losses by ko/tko,
5. wins by decision,
6. losses by decision,
7. draws

for both "Fighter A" and "Fighter B".

For additional info we will connect:
1. nationality,
2. birthday,
3. referee

for both "Fighter A" and "Fighter B".


In [5]:
records = pd.read_csv("records_clean.csv")
bouts = pd.read_csv("bouts_new.csv")

In [6]:
records.head(5)

,Fighter,URL,Nationality,Nickname,Birthday,Height ft,Height cm,Opponent,Result,Method,Round,Time,Event,Date,Referee,Description
0,Marcin Tybura,https://www.sherdog.com/fighter/Marcin-Tybura-86928,Poland,Tybur,"Nov 9, 1985","6'3""",190.5 cm,Tai Tuivasa,win,Submission,1,4:08,UFC Fight Night 239 - Tuivasa vs. Tybura,Mar / 16 / 2024,Herb Dean,Rear-Naked Choke
1,Marcin Tybura,https://www.sherdog.com/fighter/Marcin-Tybura-86928,Poland,Tybur,"Nov 9, 1985","6'3""",190.5 cm,Tom Aspinall,loss,KO/TKO,1,1:13,UFC Fight Night 224 - Aspinall vs. Tybura,Jul / 22 / 2023,Marc Goddard,Elbow and Punches
2,Marcin Tybura,https://www.sherdog.com/fighter/Marcin-Tybura-86928,Poland,Tybur,"Nov 9, 1985","6'3""",190.5 cm,Blagoy Ivanov,win,Decision,3,5:00,UFC Fight Night 218 - Lewis vs. Spivak,Feb / 04 / 2023,Mark Smith,Unanimous
3,Marcin Tybura,https://www.sherdog.com/fighter/Marcin-Tybura-86928,Poland,Tybur,"Nov 9, 1985","6'3""",190.5 cm,Alexander Romanov,win,Decision,3,5:00,UFC 278 - Usman vs. Edwards 2,Aug / 20 / 2022,Jason Herzog,Majority
4,Marcin Tybura,https://www.sherdog.com/fighter/Marcin-Tybura-86928,Poland,Tybur,"Nov 9, 1985","6'3""",190.5 cm,Alexander Volkov,loss,Decision,3,5:00,UFC 267 - Blachowicz vs. Teixeira,Oct / 30 / 2021,Marc Goddard,Unanimous


In [7]:
bouts.head()

,Bout Link,Fighter A,Link A,Fighter B,Link B,Nickname A,Nickname B,Record A,Record B,Odds A,...,Result,Time,Weightclass,Rounds,Event Link,Venue,Date,Location,Billing,Event Name
0,https://www.tapology.com/fightcenter/bouts/835883-ufc-fight-night-tai-bam-bam-tuivasa-vs-marcin-...,Marcin Tybura,https://www.tapology.com/fightcenter/fighters/34449-marcin-tybura-tybur,Tai Tuivasa,https://www.tapology.com/fightcenter/fighters/120884-tai-tuivasa-bam-bam,NaN,NaN,25-8,15-7,-105 (Near Even),...,"Submission, Rear Naked Choke",4:08 Round 1 of 5,265.0,5 x 5,https://www.tapology.com/fightcenter/events/108288-ufc-fight-night,UFC Apex,Saturday 03.16.2024 at 04:00 PM ET,"Las Vegas, Nevada, United States",Main Event,UFC Fight Night: Tuivasa vs. Tybura
1,https://www.tapology.com/fightcenter/bouts/835212-ufc-fight-night-bryan-pooh-bear-battle-vs-ange...,Bryan Battle,https://www.tapology.com/fightcenter/fighters/153353-bryan-battle,Ange Loosa,https://www.tapology.com/fightcenter/fighters/79954-ange-loosa,NaN,NaN,10-2-0,10-3-0,-180 (Slight Favorite),...,"Ends in a No Contest, Accidental Eye Poke","1:00 Round 2 of 3, 6:00 Total",170.0,3 x 5,https://www.tapology.com/fightcenter/events/108288-ufc-fight-night,UFC Apex,Saturday 03.16.2024 at 04:00 PM ET,"Las Vegas, Nevada, United States",Co-Main,UFC Fight Night: Tuivasa vs. Tybura
2,https://www.tapology.com/fightcenter/bouts/836823-ufc-fight-night-kennedy-african-savage-nzechuk...,Ovince St. Preux,https://www.tapology.com/fightcenter/fighters/ovince-st-preux,Kennedy Nzechukwu,https://www.tapology.com/fightcenter/fighters/119288-kennedy-nzechukwu,NaN,NaN,27-17,12-5,+500 (Heavy Underdog),...,"Decision, Split","3 Rounds, 15:00 Total",205.0,3 x 5,https://www.tapology.com/fightcenter/events/108288-ufc-fight-night,UFC Apex,Saturday 03.16.2024 at 04:00 PM ET,"Las Vegas, Nevada, United States",Main Card,UFC Fight Night: Tuivasa vs. Tybura
3,https://www.tapology.com/fightcenter/bouts/834662-ufc-fight-night-christian-ceerod-rodriguez-vs-...,Christian Rodriguez,https://www.tapology.com/fightcenter/fighters/121190-christian-rodriguez,Isaac Dulgarian,https://www.tapology.com/fightcenter/fighters/245772-isaac-dulgarian,NaN,NaN,11-1,6-1,+165 (Slight Underdog),...,"Decision, Split","3 Rounds, 15:00 Total",145.0,3 x 5,https://www.tapology.com/fightcenter/events/108288-ufc-fight-night,UFC Apex,Saturday 03.16.2024 at 04:00 PM ET,"Las Vegas, Nevada, United States",Main Card,UFC Fight Night: Tuivasa vs. Tybura
4,https://www.tapology.com/fightcenter/bouts/838614-ufc-fight-night-pannie-banzai-kianzad-vs-macy-...,Macy Chiasson,https://www.tapology.com/fightcenter/fighters/77737-macy-chiasson,Pannie Kianzad,https://www.tapology.com/fightcenter/fighters/36969-pannie-kianzad,NaN,NaN,9-3,16-8,-230 (Moderate Favorite),...,"Submission, Rear Naked Choke",3:54 Round 1 of 3,135.0,3 x 5,https://www.tapology.com/fightcenter/events/108288-ufc-fight-night,UFC Apex,Saturday 03.16.2024 at 04:00 PM ET,"Las Vegas, Nevada, United States",Main Card,UFC Fight Night: Tuivasa vs. Tybura


### We will calculate the current records based on fighter names and dates. Names are already synchronized, now we need to convert dates to the same format.

In [8]:
from datetime import datetime

In [9]:
# Function to convert date string to YYYY-MM-DD format for records
def convert_records_date(date_string):
    date_object = datetime.strptime(date_string, "%b / %d / %Y")
    return date_object.strftime("%Y-%m-%d")

# Apply the function to the DataFrame column
records['Date'] = records['Date'].apply(convert_records_date)

In [10]:
records.head()

,Fighter,URL,Nationality,Nickname,Birthday,Height ft,Height cm,Opponent,Result,Method,Round,Time,Event,Date,Referee,Description
0,Marcin Tybura,https://www.sherdog.com/fighter/Marcin-Tybura-86928,Poland,Tybur,"Nov 9, 1985","6'3""",190.5 cm,Tai Tuivasa,win,Submission,1,4:08,UFC Fight Night 239 - Tuivasa vs. Tybura,2024-03-16,Herb Dean,Rear-Naked Choke
1,Marcin Tybura,https://www.sherdog.com/fighter/Marcin-Tybura-86928,Poland,Tybur,"Nov 9, 1985","6'3""",190.5 cm,Tom Aspinall,loss,KO/TKO,1,1:13,UFC Fight Night 224 - Aspinall vs. Tybura,2023-07-22,Marc Goddard,Elbow and Punches
2,Marcin Tybura,https://www.sherdog.com/fighter/Marcin-Tybura-86928,Poland,Tybur,"Nov 9, 1985","6'3""",190.5 cm,Blagoy Ivanov,win,Decision,3,5:00,UFC Fight Night 218 - Lewis vs. Spivak,2023-02-04,Mark Smith,Unanimous
3,Marcin Tybura,https://www.sherdog.com/fighter/Marcin-Tybura-86928,Poland,Tybur,"Nov 9, 1985","6'3""",190.5 cm,Alexander Romanov,win,Decision,3,5:00,UFC 278 - Usman vs. Edwards 2,2022-08-20,Jason Herzog,Majority
4,Marcin Tybura,https://www.sherdog.com/fighter/Marcin-Tybura-86928,Poland,Tybur,"Nov 9, 1985","6'3""",190.5 cm,Alexander Volkov,loss,Decision,3,5:00,UFC 267 - Blachowicz vs. Teixeira,2021-10-30,Marc Goddard,Unanimous


In [11]:
# Function to convert date string to YYYY-MM-DD format for bouts
def convert_records_date(date_string):
    date_string = date_string.split(" ")[1]
    date_object = datetime.strptime(date_string, "%m.%d.%Y")
    return date_object.strftime("%Y-%m-%d")

# Apply the function to the DataFrame column
bouts['Date'] = bouts['Date'].apply(convert_records_date)

In [12]:
bouts.head()

,Bout Link,Fighter A,Link A,Fighter B,Link B,Nickname A,Nickname B,Record A,Record B,Odds A,...,Result,Time,Weightclass,Rounds,Event Link,Venue,Date,Location,Billing,Event Name
0,https://www.tapology.com/fightcenter/bouts/835883-ufc-fight-night-tai-bam-bam-tuivasa-vs-marcin-...,Marcin Tybura,https://www.tapology.com/fightcenter/fighters/34449-marcin-tybura-tybur,Tai Tuivasa,https://www.tapology.com/fightcenter/fighters/120884-tai-tuivasa-bam-bam,NaN,NaN,25-8,15-7,-105 (Near Even),...,"Submission, Rear Naked Choke",4:08 Round 1 of 5,265.0,5 x 5,https://www.tapology.com/fightcenter/events/108288-ufc-fight-night,UFC Apex,2024-03-16,"Las Vegas, Nevada, United States",Main Event,UFC Fight Night: Tuivasa vs. Tybura
1,https://www.tapology.com/fightcenter/bouts/835212-ufc-fight-night-bryan-pooh-bear-battle-vs-ange...,Bryan Battle,https://www.tapology.com/fightcenter/fighters/153353-bryan-battle,Ange Loosa,https://www.tapology.com/fightcenter/fighters/79954-ange-loosa,NaN,NaN,10-2-0,10-3-0,-180 (Slight Favorite),...,"Ends in a No Contest, Accidental Eye Poke","1:00 Round 2 of 3, 6:00 Total",170.0,3 x 5,https://www.tapology.com/fightcenter/events/108288-ufc-fight-night,UFC Apex,2024-03-16,"Las Vegas, Nevada, United States",Co-Main,UFC Fight Night: Tuivasa vs. Tybura
2,https://www.tapology.com/fightcenter/bouts/836823-ufc-fight-night-kennedy-african-savage-nzechuk...,Ovince St. Preux,https://www.tapology.com/fightcenter/fighters/ovince-st-preux,Kennedy Nzechukwu,https://www.tapology.com/fightcenter/fighters/119288-kennedy-nzechukwu,NaN,NaN,27-17,12-5,+500 (Heavy Underdog),...,"Decision, Split","3 Rounds, 15:00 Total",205.0,3 x 5,https://www.tapology.com/fightcenter/events/108288-ufc-fight-night,UFC Apex,2024-03-16,"Las Vegas, Nevada, United States",Main Card,UFC Fight Night: Tuivasa vs. Tybura
3,https://www.tapology.com/fightcenter/bouts/834662-ufc-fight-night-christian-ceerod-rodriguez-vs-...,Christian Rodriguez,https://www.tapology.com/fightcenter/fighters/121190-christian-rodriguez,Isaac Dulgarian,https://www.tapology.com/fightcenter/fighters/245772-isaac-dulgarian,NaN,NaN,11-1,6-1,+165 (Slight Underdog),...,"Decision, Split","3 Rounds, 15:00 Total",145.0,3 x 5,https://www.tapology.com/fightcenter/events/108288-ufc-fight-night,UFC Apex,2024-03-16,"Las Vegas, Nevada, United States",Main Card,UFC Fight Night: Tuivasa vs. Tybura
4,https://www.tapology.com/fightcenter/bouts/838614-ufc-fight-night-pannie-banzai-kianzad-vs-macy-...,Macy Chiasson,https://www.tapology.com/fightcenter/fighters/77737-macy-chiasson,Pannie Kianzad,https://www.tapology.com/fightcenter/fighters/36969-pannie-kianzad,NaN,NaN,9-3,16-8,-230 (Moderate Favorite),...,"Submission, Rear Naked Choke",3:54 Round 1 of 3,135.0,3 x 5,https://www.tapology.com/fightcenter/events/108288-ufc-fight-night,UFC Apex,2024-03-16,"Las Vegas, Nevada, United States",Main Card,UFC Fight Night: Tuivasa vs. Tybura


Dates in both tables now have the same format.

In [13]:
df = records.loc[records["Fighter"] == "Marcin Tybura"]
df.loc[df["Date"] == "2024-03-16", "Referee"]

0    Herb Dean
Name: Referee, dtype: object

### Calculating the records

We define a function that calculates required info and returns it in a list

In [50]:
def calculate_records(fighter_name, bout_date):
    df = records.loc[records["Fighter"] == fighter_name]
    # Finds fighter information
    nationality, nickname, birthday, referee = "", "", "", ""
    nationality = df.iloc[0]["Nationality"]
    nickname = df.iloc[0]["Nickname"]
    birthday = df.iloc[0]["Birthday"]
    if not df.loc[df["Date"] == bout_date, "Referee"].empty:
        referee = df.loc[df["Date"] == bout_date, "Referee"].iloc[0]
    # Filters DataFrame for the date of a given fight
    df = df.loc[df["Date"] < bout_date]
    # Calculates fighters record
    counts = df.groupby(["Result","Method"])["Fighter"].count()
    ko_win, ko_loss, sub_win, sub_loss, dec_win, dec_loss, draws, ufc_win, ufc_loss, ufc_draws = 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
    for index, value in counts.items():
        if index[0] + index[1] == "winKO/TKO":
            ko_win = value
        elif index[0] + index[1] == "lossKO/TKO":
            ko_loss = value  
        elif index[0] + index[1] == "winSubmission":
            sub_win = value                  
        elif index[0] + index[1] == "lossSubmission":
            sub_loss = value
        elif index[0] + index[1] == "winDecision":
            dec_win = value                 
        elif index[0] + index[1] == "lossDecision":
            dec_loss = value      
        elif index[0] == "draw":
            draws = value
    
    # Calculates fighters ufc record
    ufc_events = df[df['Event'].str.startswith('UFC')]
    ufc_counts = ufc_events.groupby("Result")["Fighter"].count()
    for index, value in ufc_counts.items():
        if index == "win":
            ufc_win = value 
        elif index == "loss":
            ufc_loss = value
        elif index == "draw":
            ufc_draws = value
    info_list=[]
    info_list = [ko_win, ko_loss, sub_win, sub_loss, dec_win, dec_loss, draws, ufc_win, ufc_loss, ufc_draws, nationality, birthday, nickname, referee]
    return info_list

We call the function for "Fighter A" and "Fighter B" for all bouts and store info in dataframes

In [48]:
from tqdm import tqdm

In [49]:
info_dfA = pd.DataFrame(columns=["KO win A", "KO loss A", "Sub win A", "Sub loss A", "Dec win A", "Dec loss A", 
                                 "Draws A", "UFC win A", "UFC loss A", "UFC draws A", "Nationality A", "Birthday A", "Nick A", "Referee"])
info_dfB = pd.DataFrame(columns=["KO win B", "KO loss B", "Sub win B", "Sub loss B", "Dec win B", "Dec loss B", 
                                 "Draws B", "UFC win B", "UFC loss B", "UFC draws B", "Nationality B", "Birthday B", "Nick B", "Referee B"])
for i in tqdm(bouts.index):
        info_list = calculate_records(bouts["Fighter A"][i], bouts["Date"][i])
        info_dfA.loc[len(info_dfA)] = info_list
        info_list = calculate_records(bouts["Fighter B"][i], bouts["Date"][i])
        info_dfB.loc[len(info_dfB)] = info_list

100%|██████████████████████████████████████████████████████████████████████████████| 7577/7577 [02:04<00:00, 60.85it/s]


In [51]:
info_dfA.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7577 entries, 0 to 7576
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   KO win A       7577 non-null   int64 
 1   KO loss A      7577 non-null   int64 
 2   Sub win A      7577 non-null   int64 
 3   Sub loss A     7577 non-null   int64 
 4   Dec win A      7577 non-null   int64 
 5   Dec loss A     7577 non-null   int64 
 6   Draws A        7577 non-null   int64 
 7   UFC win A      7577 non-null   int64 
 8   UFC loss A     7577 non-null   int64 
 9   UFC draws A    7577 non-null   int64 
 10  Nationality A  7576 non-null   object
 11  Birthday A     7545 non-null   object
 12  Nick A         5945 non-null   object
 13  Referee        7577 non-null   object
dtypes: int64(10), object(4)
memory usage: 887.9+ KB


In [52]:
info_dfB.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7577 entries, 0 to 7576
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   KO win B       7577 non-null   int64 
 1   KO loss B      7577 non-null   int64 
 2   Sub win B      7577 non-null   int64 
 3   Sub loss B     7577 non-null   int64 
 4   Dec win B      7577 non-null   int64 
 5   Dec loss B     7577 non-null   int64 
 6   Draws B        7577 non-null   int64 
 7   UFC win B      7577 non-null   int64 
 8   UFC loss B     7577 non-null   int64 
 9   UFC draws B    7577 non-null   int64 
 10  Nationality B  7573 non-null   object
 11  Birthday B     7454 non-null   object
 12  Nick B         5826 non-null   object
 13  Referee B      7577 non-null   object
dtypes: int64(10), object(4)
memory usage: 887.9+ KB


## Now we can finally merge bouts with info about "Fighter A" and "Fighter B"

In [53]:
combined_df = pd.concat([bouts, info_dfA, info_dfB], axis=1)

In [54]:
combined_df.to_csv("bouts_detailed.csv", index=False)

In [55]:
combined_df = pd.read_csv("bouts_detailed.csv")

In [56]:
combined_df.columns

Index(['Bout Link', 'Fighter A', 'Link A', 'Fighter B', 'Link B', 'Nickname A',
       'Nickname B', 'Record A', 'Record B', 'Odds A', 'Odds B', 'Title A',
       'Title B', 'Weight A', 'Weight B', 'Age A', 'Age B', 'Height A',
       'Height B', 'Reach A', 'Reach B', 'Result', 'Time', 'Weightclass',
       'Rounds', 'Event Link', 'Venue', 'Date', 'Location', 'Billing',
       'Event Name', 'KO win A', 'KO loss A', 'Sub win A', 'Sub loss A',
       'Dec win A', 'Dec loss A', 'Draws A', 'UFC win A', 'UFC loss A',
       'UFC draws A', 'Nationality A', 'Birthday A', 'Nick A', 'Referee',
       'KO win B', 'KO loss B', 'Sub win B', 'Sub loss B', 'Dec win B',
       'Dec loss B', 'Draws B', 'UFC win B', 'UFC loss B', 'UFC draws B',
       'Nationality B', 'Birthday B', 'Nick B', 'Referee B'],
      dtype='object')

# Data wrangling

We have finally obtained a dataset that contains all needed information. Now we have to clean it, transform it and replace some values.


In [3]:
df = pd.read_csv("bouts_detailed.csv")

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7577 entries, 0 to 7576
Data columns (total 59 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Bout Link      7577 non-null   object 
 1   Fighter A      7577 non-null   object 
 2   Link A         7577 non-null   object 
 3   Fighter B      7577 non-null   object 
 4   Link B         7577 non-null   object 
 5   Nickname A     0 non-null      float64
 6   Nickname B     0 non-null      float64
 7   Record A       7577 non-null   object 
 8   Record B       7577 non-null   object 
 9   Odds A         7129 non-null   object 
 10  Odds B         7128 non-null   object 
 11  Title A        347 non-null    object 
 12  Title B        347 non-null    object 
 13  Weight A       7509 non-null   object 
 14  Weight B       7492 non-null   object 
 15  Age A          7539 non-null   object 
 16  Age B          7440 non-null   object 
 17  Height A       7576 non-null   object 
 18  Height B

In [58]:
df.head()

,Bout Link,Fighter A,Link A,Fighter B,Link B,Nickname A,Nickname B,Record A,Record B,Odds A,...,Dec win B,Dec loss B,Draws B,UFC win B,UFC loss B,UFC draws B,Nationality B,Birthday B,Nick B,Referee B
0,https://www.tapology.com/fightcenter/bouts/835883-ufc-fight-night-tai-bam-bam-tuivasa-vs-marcin-...,Marcin Tybura,https://www.tapology.com/fightcenter/fighters/34449-marcin-tybura-tybur,Tai Tuivasa,https://www.tapology.com/fightcenter/fighters/120884-tai-tuivasa-bam-bam,NaN,NaN,25-8,15-7,-105 (Near Even),...,1,1,0,8,6,0,Australia,"Mar 16, 1993",Bam Bam,Herb Dean
1,https://www.tapology.com/fightcenter/bouts/835212-ufc-fight-night-bryan-pooh-bear-battle-vs-ange...,Bryan Battle,https://www.tapology.com/fightcenter/fighters/153353-bryan-battle,Ange Loosa,https://www.tapology.com/fightcenter/fighters/79954-ange-loosa,NaN,NaN,10-2-0,10-3-0,-180 (Slight Favorite),...,4,3,0,2,1,0,"Congo, The Democratic Republic of the","Mar 18, 1993",The Last Ninja,Mike Beltran
2,https://www.tapology.com/fightcenter/bouts/836823-ufc-fight-night-kennedy-african-savage-nzechuk...,Ovince St. Preux,https://www.tapology.com/fightcenter/fighters/ovince-st-preux,Kennedy Nzechukwu,https://www.tapology.com/fightcenter/fighters/119288-kennedy-nzechukwu,NaN,NaN,27-17,12-5,+500 (Heavy Underdog),...,3,1,0,6,4,0,United States,"Jun 13, 1992",American Savage,Mark Smith
3,https://www.tapology.com/fightcenter/bouts/834662-ufc-fight-night-christian-ceerod-rodriguez-vs-...,Christian Rodriguez,https://www.tapology.com/fightcenter/fighters/121190-christian-rodriguez,Isaac Dulgarian,https://www.tapology.com/fightcenter/fighters/245772-isaac-dulgarian,NaN,NaN,11-1,6-1,+165 (Slight Underdog),...,0,0,0,1,0,0,United States,"Jul 4, 1996",The MidWest Choppa,Chris Tognoni
4,https://www.tapology.com/fightcenter/bouts/838614-ufc-fight-night-pannie-banzai-kianzad-vs-macy-...,Macy Chiasson,https://www.tapology.com/fightcenter/fighters/77737-macy-chiasson,Pannie Kianzad,https://www.tapology.com/fightcenter/fighters/36969-pannie-kianzad,NaN,NaN,9-3,16-8,-230 (Moderate Favorite),...,13,4,0,5,4,0,Sweden,"Dec 8, 1991",Banzai,Mark Smith


In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7577 entries, 0 to 7576
Data columns (total 59 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Bout Link      7577 non-null   object 
 1   Fighter A      7577 non-null   object 
 2   Link A         7577 non-null   object 
 3   Fighter B      7577 non-null   object 
 4   Link B         7577 non-null   object 
 5   Nickname A     0 non-null      float64
 6   Nickname B     0 non-null      float64
 7   Record A       7577 non-null   object 
 8   Record B       7577 non-null   object 
 9   Odds A         7129 non-null   object 
 10  Odds B         7128 non-null   object 
 11  Title A        347 non-null    object 
 12  Title B        347 non-null    object 
 13  Weight A       7509 non-null   object 
 14  Weight B       7492 non-null   object 
 15  Age A          7539 non-null   object 
 16  Age B          7440 non-null   object 
 17  Height A       7576 non-null   object 
 18  Height B

We can see that there is a 53 columns, most of them are doubled for fighter A and B. To avoid the confusion we can change the order so they are together.

In [60]:
columns = df.columns
new_columns = (['Bout Link', 'Fighter A', 'Fighter B'] + 
               [col for col in columns if col not in ['Bout Link', 'Fighter A', 'Fighter B'] and not col.endswith((' A', ' B'))] + 
               [col for col in columns if col.endswith(' A') and col not in ['Fighter A']] +
               [col for col in columns if col.endswith(' B') and col not in ['Fighter B']])
print(new_columns, "\nNumber of cols:", len(new_columns))

['Bout Link', 'Fighter A', 'Fighter B', 'Result', 'Time', 'Weightclass', 'Rounds', 'Event Link', 'Venue', 'Date', 'Location', 'Billing', 'Event Name', 'Referee', 'Link A', 'Nickname A', 'Record A', 'Odds A', 'Title A', 'Weight A', 'Age A', 'Height A', 'Reach A', 'KO win A', 'KO loss A', 'Sub win A', 'Sub loss A', 'Dec win A', 'Dec loss A', 'Draws A', 'UFC win A', 'UFC loss A', 'UFC draws A', 'Nationality A', 'Birthday A', 'Nick A', 'Link B', 'Nickname B', 'Record B', 'Odds B', 'Title B', 'Weight B', 'Age B', 'Height B', 'Reach B', 'KO win B', 'KO loss B', 'Sub win B', 'Sub loss B', 'Dec win B', 'Dec loss B', 'Draws B', 'UFC win B', 'UFC loss B', 'UFC draws B', 'Nationality B', 'Birthday B', 'Nick B', 'Referee B'] 
Number of cols: 59


After visual check we see that the order is how we want it and there is the same amount of columns.

Now we just change it in the dataframe.

In [61]:
df = df[new_columns]

In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7577 entries, 0 to 7576
Data columns (total 59 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Bout Link      7577 non-null   object 
 1   Fighter A      7577 non-null   object 
 2   Fighter B      7577 non-null   object 
 3   Result         7577 non-null   object 
 4   Time           7577 non-null   object 
 5   Weightclass    7453 non-null   float64
 6   Rounds         7463 non-null   object 
 7   Event Link     7577 non-null   object 
 8   Venue          7577 non-null   object 
 9   Date           7577 non-null   object 
 10  Location       7577 non-null   object 
 11  Billing        7577 non-null   object 
 12  Event Name     7577 non-null   object 
 13  Referee        7471 non-null   object 
 14  Link A         7577 non-null   object 
 15  Nickname A     0 non-null      float64
 16  Record A       7577 non-null   object 
 17  Odds A         7129 non-null   object 
 18  Title A 

Lets drop "Nickname A" and "Nickname B" since they have no values, we will leave "Nick A" and "Nick B" instead.

In [63]:
df.drop(['Nickname A', 'Nickname B'], axis=1, inplace=True)

Extract the "Description" from the "Result" and merge results into categories

In [64]:
df['Description'] = df['Result'].str.extract(r',\s*(.*)', expand=False)
df["Result"].replace(r',\s*(.*)', "", regex=True, inplace=True)

In [65]:
df["Result"].value_counts()

Result
Decision                    3416
KO/TKO                      2451
Submission                  1546
Ends in a No Contest          66
Ends in a Draw                58
Disqualificaton               20
Overturned to No Contest      19
Result Overturned              1
Name: count, dtype: int64

In [66]:
# 1
df.loc[df["Result"] == "Ends in a No Contest", "Result"] = "No Contest"
df.loc[df["Result"] == "Overturned to No Contest", "Result"] = "No Contest"
df.loc[df["Result"] == "Result Overturned", "Result"] = "No Contest"
# 2
df.loc[df["Result"] == "Disqualificaton", "Result"] = "KO/TKO"
# 3
df.loc[df["Result"] == "Ends in a Draw", "Result"] = "Draw"

In [67]:
df["Result"].value_counts()

Result
Decision      3416
KO/TKO        2471
Submission    1546
No Contest      86
Draw            58
Name: count, dtype: int64

The "Result" column in this dataset is more of a "Method" how the fight ended. "Fighter A" column is a winner and "Fighter B" is a loser unless the fight ends in a "No Contest" or a "Draw".

We have to create new column "Method", copy the data from "Result" and fill the "Result" with real result data(win, loss, no contest, draw).

In [68]:
df["Method"] = df["Result"]

In [69]:
for i in df.index:
    if df.loc[i, "Method"] in ["Decision", "KO/TKO", "Submission"]:
        df.loc[i, "Result"] = "win"
    elif df.loc[i, "Method"] == "No Contest":
        df.loc[i, "Result"] = "NC"
    elif df.loc[i, "Method"] == "Draw":
        df.loc[i, "Result"] = "draw"

In [70]:
df["Result"].value_counts()

Result
win     7433
NC        86
draw      58
Name: count, dtype: int64

We have successfully created new column "Method" and filled the "Result" column with real result data(win, loss, NC, draw).

Now we can change the "Draw" values in "Method" column to decision.

In [71]:
df.loc[df["Method"] == "Draw", "Method"] = "Decision"

In [72]:
df["Method"].value_counts()

Method
Decision      3474
KO/TKO        2471
Submission    1546
No Contest      86
Name: count, dtype: int64

We will take a closer look at the "Time" column since it is not structured.

In [73]:
df[["Time", "Result", "Method"]].head()

,Time,Result,Method
0,4:08 Round 1 of 5,win,Submission
1,"1:00 Round 2 of 3, 6:00 Total",NC,No Contest
2,"3 Rounds, 15:00 Total",win,Decision
3,"3 Rounds, 15:00 Total",win,Decision
4,3:54 Round 1 of 3,win,Submission


There are two forms of records based on, the "Method" fight ended. 

If it ended on "Decision" and "No Contest", or if it ended on "KO/TKO" and "Submission".

We will transform this column to show what round did the fight prematurely finish.

For "KO/TKO" and "Submission" there will be values 1 to 5.

For "Decision" and "No Contest" there will be value 0 as the fight wasnt prematurely finished. 

In [74]:
for i in df.index:
    if df.loc[i, "Method"] in ["KO/TKO", "Submission"]:
        df.loc[i, "Finish"] = df.loc[i, 'Time'].split("Round ")[1].split(" ")[0]
    elif df.loc[i, "Method"] in ["Decision", "No Contest"]:
        df.loc[i, "Finish"] = "0"  

C:\Users\Uživatel\AppData\Local\Temp\ipykernel_25140\2166255254.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '1' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[i, "Finish"] = df.loc[i, 'Time'].split("Round ")[1].split(" ")[0]


In [75]:
df["Finish"].value_counts()

Finish
0    3560
1    2145
2    1209
3     590
4      43
5      30
Name: count, dtype: int64

Now we try to add missing values in column "Referee" with values from "Referee B" and drop the second.

In [76]:
df.loc[(df["Referee"].isna()) & df["Referee B"].notna() , "Referee"] = df["Referee B"]

In [77]:
df.drop("Referee B", axis=1, inplace = True)

For height and reach we will use values in centimeters so we have to extract them.

In [78]:
df[["Height A", "Height B", "Reach A", "Reach B"]].head()

,Height A,Height B,Reach A,Reach B
0,"6'3"" (191cm)","6'2"" (188cm)","78.0"" (198cm)","75.0"" (191cm)"
1,"6'1"" (185cm)","5'10"" (178cm)","77.0"" (196cm)","74.0"" (188cm)"
2,"6'3"" (191cm)","6'5"" (196cm)","80.0"" (203cm)","83.0"" (211cm)"
3,"5'7"" (170cm)","5'7"" (170cm)","71.0"" (180cm)","71.0"" (180cm)"
4,"5'11"" (180cm)","5'7"" (170cm)","72.0"" (183cm)","66.0"" (168cm)"


In [79]:
df['Height A'] = df['Height A'].str.extract(r'(\d{3})cm', expand=False)
df['Height B'] = df['Height B'].str.extract(r'(\d{3})cm', expand=False)
df['Reach A'] = df['Reach A'].str.extract(r'(\d{3})cm', expand=False)
df['Reach B'] = df['Reach B'].str.extract(r'(\d{3})cm', expand=False)

In [80]:
df[["Height A", "Height B", "Reach A", "Reach B"]].head()

,Height A,Height B,Reach A,Reach B
0,191,188,198,191
1,185,178,196,188
2,191,196,203,211
3,170,170,180,180
4,180,170,183,168


For weight we will use values in pounds so we have to extract them.

In [81]:
df[['Weight A', "Weight B"]].head()

,Weight A,Weight B
0,247.0 lbs (112.0 kgs),265.5 lbs (120.4 kgs)
1,170.0 lbs (77.1 kgs),171.0 lbs (77.6 kgs)
2,206.0 lbs (93.4 kgs),205.0 lbs (93.0 kgs)
3,145.0 lbs (65.8 kgs),145.5 lbs (66.0 kgs)
4,135.5 lbs (61.5 kgs),136.0 lbs (61.7 kgs)


In [82]:
df['Weight A'] = df['Weight A'].str.extract(r'([\d.]+) lbs', expand=False)
df['Weight B'] = df['Weight B'].str.extract(r'([\d.]+) lbs', expand=False)

In [83]:
df[['Weight A', "Weight B"]].head()

,Weight A,Weight B
0,247.0,265.5
1,170.0,171.0
2,206.0,205.0
3,145.0,145.5
4,135.5,136.0


For age we extract only years.

In [84]:
df[["Age A", "Age B"]].head()

,Age A,Age B
0,"38 years, 4 months, 1 week",31 years
1,"29 years, 5 months, 3 weeks, 3 days","29 years, 11 months, 3 weeks, 6 days"
2,"40 years, 11 months, 1 week, 1 day","31 years, 9 months, 3 days"
3,"26 years, 2 months, 4 weeks","27 years, 8 months, 1 week, 5 days"
4,"32 years, 7 months, 2 weeks, 4 days","32 years, 3 months, 1 week, 1 day"


In [85]:
df['Age A'] = df['Age A'].str.extract(r'(\d+) years', expand=False)
df['Age B'] = df['Age B'].str.extract(r'(\d+) years', expand=False)

In [86]:
df[["Age A", "Age B"]].head()

,Age A,Age B
0,38,31
1,29,29
2,40,31
3,26,27
4,32,32


We split odds for numerical value and category.

In [87]:
df[["Odds A", "Odds B"]].head()

,Odds A,Odds B
0,-105 (Near Even),-115 (Near Even)
1,-180 (Slight Favorite),+150 (Slight Underdog)
2,+500 (Heavy Underdog),-700 (Huge Favorite)
3,+165 (Slight Underdog),-200 (Moderate Favorite)
4,-230 (Moderate Favorite),+190 (Slight Underdog)


In [88]:
df['Odds cat A'] = df['Odds A'].str.extract(r'\((.*?)\)', expand=False)
df["Odds A"].replace(r'\((.*?)\)', "", regex=True, inplace=True)

In [89]:
df['Odds cat B'] = df['Odds B'].str.extract(r'\((.*?)\)', expand=False)
df["Odds B"].replace(r'\((.*?)\)', "", regex=True, inplace=True)

In [90]:
df[["Odds A", "Odds cat A", "Odds B", "Odds cat B"]].head()

,Odds A,Odds cat A,Odds B,Odds cat B
0,-105,Near Even,-115,Near Even
1,-180,Slight Favorite,+150,Slight Underdog
2,+500,Heavy Underdog,-700,Huge Favorite
3,+165,Slight Underdog,-200,Moderate Favorite
4,-230,Moderate Favorite,+190,Slight Underdog


## Drop non valuable columns for further analysis

For further analysis there are some useless columns that we can drop:
1. "Bout Link", 
2. "Time" - we already stored the important information in "Finish",
3. "Event Link" - we will keep "Event Name" which has same purpose,
4. "Link A" and "Link B",
5. "Record A", "Record B" - we have detailed record in other columns,
6. "Birthday A" and "Birthday B" - we keep the age columns which are more friendly to work with.


In [91]:
df.drop(["Bout Link", "Time", "Event Link", "Link A", "Link B","Record A", "Record B", "Birthday A", "Birthday B"], axis=1, inplace=True)

In [92]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7577 entries, 0 to 7576
Data columns (total 52 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Fighter A      7577 non-null   object 
 1   Fighter B      7577 non-null   object 
 2   Result         7577 non-null   object 
 3   Weightclass    7453 non-null   float64
 4   Rounds         7463 non-null   object 
 5   Venue          7577 non-null   object 
 6   Date           7577 non-null   object 
 7   Location       7577 non-null   object 
 8   Billing        7577 non-null   object 
 9   Event Name     7577 non-null   object 
 10  Referee        7475 non-null   object 
 11  Odds A         7129 non-null   object 
 12  Title A        347 non-null    object 
 13  Weight A       7509 non-null   object 
 14  Age A          7539 non-null   object 
 15  Height A       7576 non-null   object 
 16  Reach A        7181 non-null   object 
 17  KO win A       7577 non-null   int64  
 18  KO loss 

## Drop rows of bouts before the adoption of NJSACB rules

In [93]:
df = df.loc[df["Date"] > "2001-04-03"]

In [94]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7294 entries, 0 to 7293
Data columns (total 52 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Fighter A      7294 non-null   object 
 1   Fighter B      7294 non-null   object 
 2   Result         7294 non-null   object 
 3   Weightclass    7294 non-null   float64
 4   Rounds         7225 non-null   object 
 5   Venue          7294 non-null   object 
 6   Date           7294 non-null   object 
 7   Location       7294 non-null   object 
 8   Billing        7294 non-null   object 
 9   Event Name     7294 non-null   object 
 10  Referee        7192 non-null   object 
 11  Odds A         7126 non-null   object 
 12  Title A        328 non-null    object 
 13  Weight A       7271 non-null   object 
 14  Age A          7290 non-null   object 
 15  Height A       7294 non-null   object 
 16  Reach A        7125 non-null   object 
 17  KO win A       7294 non-null   int64  
 18  KO loss A    

We dropped 283 rows of bouts before the adoption of NJSACB rules. (wikipedia)

## Dealing with missing values

### Rounds

We can start by dropping all the rows that are not a modern professional MMA bout based on the "Rounds" column. Everything that is not "3 x 5" or "5 x 5" needs to be dropped.

In [95]:
df["Rounds"].value_counts()

Rounds
3 x 5    6570
5 x 5     653
3 x 3       2
Name: count, dtype: int64

We can check those two values of "3 x 3" manually.

In [96]:
df.loc[df["Rounds"] == "3 x 3"]

,Fighter A,Fighter B,Result,Weightclass,Rounds,Venue,Date,Location,Billing,Event Name,...,UFC win B,UFC loss B,UFC draws B,Nationality B,Nick B,Description,Method,Finish,Odds cat A,Odds cat B
5252,Felipe Arantes,Godofredo Pepey,win,145.0,3 x 3,Ginásio Paulo Sarasate,2013-06-08,"Fortaleza, Ceara, Brazil",Prelim,UFC on FUEL TV 10: Nogueira vs. Werdum 2,...,1,1,0,Brazil,Pepey,Punches and Elbows,KO/TKO,1,Moderate Favorite,Slight Underdog
5313,Clint Hester,Bristol Marunde,win,185.0,3 x 3,Mandalay Bay Events Center,2013-04-13,"Las Vegas, Nevada, United States",Prelim,The Ultimate Fighter 17 Finale,...,0,0,0,United States,NaN,Elbow,KO/TKO,3,Slight Favorite,Slight Underdog


In [97]:
df.loc[df["Rounds"] == "3 x 3", "Rounds"] = "3 x 5"

After manual investigestion, we discovered that in both cases the bouts were for 3 x 5 rounds not 3 x 3, so we changed them.

Now we have to deal with missing values. 

All bouts have to be "3 x 5" or "5 x 5", basic standard for UFC is that a title fights or a main events are "5 x 5" and the rest is "3 x 5".

In [98]:
for i in df.index:
    if pd.isnull(df.loc[i, "Rounds"]):
        if pd.notnull(df.loc[i, "Title A"]) or df.loc[i, "Billing"] == "Main Event":
            df.loc[i, "Rounds"] = "5 x 5"
        else:
            df.loc[i, "Rounds"] = "3 x 5"

In [99]:
df["Rounds"].value_counts()

Rounds
3 x 5    6634
5 x 5     660
Name: count, dtype: int64

We can see that of the 69 null values 7 were "5 x 5" and the rest was "3 x 5".
Rounds columns is now full of values.

For future manipulation it would be easier, if we kept just the number of rounds. Since all rounds are 5 minutes we dont need to store it in this format.

In [100]:
df.loc[df["Rounds"] == "3 x 5", "Rounds"] = "3"
df.loc[df["Rounds"] == "5 x 5", "Rounds"] = "5"

In [101]:
df["Rounds"].value_counts()

Rounds
3    6634
5     660
Name: count, dtype: int64

### Odds

We will replace all missing values with "+100" and "Even".

In [102]:
df.loc[df["Odds A"].isnull(), "Odds A"] = "+100"
df.loc[df["Odds B"].isnull(), "Odds B"] = "+100"
df.loc[df["Odds cat A"].isnull(), "Odds cat A"] = "Even"
df.loc[df["Odds cat B"].isnull(), "Odds cat B"] = "Even"

Values in 168 for A resp. 169 for B rows were replaced by even odds.

In [103]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7294 entries, 0 to 7293
Data columns (total 52 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Fighter A      7294 non-null   object 
 1   Fighter B      7294 non-null   object 
 2   Result         7294 non-null   object 
 3   Weightclass    7294 non-null   float64
 4   Rounds         7294 non-null   object 
 5   Venue          7294 non-null   object 
 6   Date           7294 non-null   object 
 7   Location       7294 non-null   object 
 8   Billing        7294 non-null   object 
 9   Event Name     7294 non-null   object 
 10  Referee        7192 non-null   object 
 11  Odds A         7294 non-null   object 
 12  Title A        328 non-null    object 
 13  Weight A       7271 non-null   object 
 14  Age A          7290 non-null   object 
 15  Height A       7294 non-null   object 
 16  Reach A        7125 non-null   object 
 17  KO win A       7294 non-null   int64  
 18  KO loss A    

### Weight

We replace weight of fighter with the value of "Weightclass" meaning they made the scheduled weight.

In [104]:
df.loc[df["Weight A"].isnull(), "Weight A"] = df["Weightclass"]
df.loc[df["Weight B"].isnull(), "Weight B"] = df["Weightclass"]

### Age


We replace missing age values with average.

In [105]:
avg_a = pd.to_numeric(df['Age A'], errors='coerce').mean()
avg_b = pd.to_numeric(df['Age B'], errors='coerce').mean()
avg_age = (avg_a + avg_b) / 2
print(avg_age)

29.766880252723


In [106]:
df.loc[df["Age A"].isnull(), "Age A"] = "30"
df.loc[df["Age B"].isnull(), "Age B"] = "30"

We found average age in the dataset rounded it to 30 and replaced the missing values.

### Height and Reach

We will replace height and reach with average in the selected "Weightclass".

In [107]:
# Convert "Height" column to numeric type
df['Height A'] = pd.to_numeric(df['Height A'], errors='coerce')

# Calculate the average height within each "Weightclass" group
average_height_by_weightclass = df.groupby('Weightclass')['Height A'].transform('mean')

# Fill missing values
df['Height A'] = df['Height A'].fillna(average_height_by_weightclass)

Now we do the same for the other columns.

In [108]:
# "Height B"
df['Height B'] = pd.to_numeric(df['Height B'], errors='coerce')
average_height_by_weightclass = df.groupby('Weightclass')['Height B'].transform('mean')
df['Height B'] = df['Height B'].fillna(average_height_by_weightclass)

# "Reach A"
df['Reach A'] = pd.to_numeric(df['Reach A'], errors='coerce')
average_height_by_weightclass = df.groupby('Weightclass')['Reach A'].transform('mean')
df['Reach A'] = df['Reach A'].fillna(average_height_by_weightclass)

# "Reach B"
df['Reach B'] = pd.to_numeric(df['Reach B'], errors='coerce')
average_height_by_weightclass = df.groupby('Weightclass')['Reach B'].transform('mean')
df['Reach B'] = df['Reach B'].fillna(average_height_by_weightclass)

In [109]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7294 entries, 0 to 7293
Data columns (total 52 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Fighter A      7294 non-null   object 
 1   Fighter B      7294 non-null   object 
 2   Result         7294 non-null   object 
 3   Weightclass    7294 non-null   float64
 4   Rounds         7294 non-null   object 
 5   Venue          7294 non-null   object 
 6   Date           7294 non-null   object 
 7   Location       7294 non-null   object 
 8   Billing        7294 non-null   object 
 9   Event Name     7294 non-null   object 
 10  Referee        7192 non-null   object 
 11  Odds A         7294 non-null   object 
 12  Title A        328 non-null    object 
 13  Weight A       7294 non-null   object 
 14  Age A          7294 non-null   object 
 15  Height A       7294 non-null   int64  
 16  Reach A        7294 non-null   float64
 17  KO win A       7294 non-null   int64  
 18  KO loss A    

In [110]:
df[df["Reach B"].isnull()]

,Fighter A,Fighter B,Result,Weightclass,Rounds,Venue,Date,Location,Billing,Event Name,...,UFC win B,UFC loss B,UFC draws B,Nationality B,Nick B,Description,Method,Finish,Odds cat A,Odds cat B
6874,Anderson Silva,Travis Lutter,win,187.0,3,Mandalay Bay Events Center,2007-02-03,"Las Vegas, Nevada, United States",Main Event,UFC 67: All or Nothing,...,2,2,0,United States,The Serial Killer,Elbows,Submission,2,Heavy Favorite,Heavy Underdog


In [111]:
avg_reach = pd.to_numeric(df.loc[df['Weightclass'] == 185, "Reach B"], errors='coerce').mean()
df.loc[df["Reach B"].isnull(), "Reach B"] = avg_reach
avg_reach

190.12674418604652

One value was still missing since there was no data for the group, so we calculated the average for the "Weightclass" close to the missing value.

In [112]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7294 entries, 0 to 7293
Data columns (total 52 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Fighter A      7294 non-null   object 
 1   Fighter B      7294 non-null   object 
 2   Result         7294 non-null   object 
 3   Weightclass    7294 non-null   float64
 4   Rounds         7294 non-null   object 
 5   Venue          7294 non-null   object 
 6   Date           7294 non-null   object 
 7   Location       7294 non-null   object 
 8   Billing        7294 non-null   object 
 9   Event Name     7294 non-null   object 
 10  Referee        7192 non-null   object 
 11  Odds A         7294 non-null   object 
 12  Title A        328 non-null    object 
 13  Weight A       7294 non-null   object 
 14  Age A          7294 non-null   object 
 15  Height A       7294 non-null   int64  
 16  Reach A        7294 non-null   float64
 17  KO win A       7294 non-null   int64  
 18  KO loss A    

Now we change the data types of columns that are numerical.

In [113]:
df.iloc[: , 40:60].head(5)

,Dec loss B,Draws B,UFC win B,UFC loss B,UFC draws B,Nationality B,Nick B,Description,Method,Finish,Odds cat A,Odds cat B
0,1,0,8,6,0,Australia,Bam Bam,Rear Naked Choke,Submission,1,Near Even,Near Even
1,3,0,2,1,0,"Congo, The Democratic Republic of the",The Last Ninja,Accidental Eye Poke,No Contest,0,Slight Favorite,Slight Underdog
2,1,0,6,4,0,United States,American Savage,Split,Decision,0,Heavy Underdog,Huge Favorite
3,0,0,1,0,0,United States,The MidWest Choppa,Split,Decision,0,Slight Underdog,Moderate Favorite
4,4,0,5,4,0,Sweden,Banzai,Rear Naked Choke,Submission,1,Moderate Favorite,Slight Underdog


In [114]:
df[["Rounds", "Odds A", "Odds B", "Age A", "Age B", "Finish"]] = df[["Rounds", "Odds A", "Odds B", "Age A", "Age B", "Finish"]].astype(int)
df[["Weight A", "Weight B", "Height A"]] = df[["Weight A", "Weight B", "Height A"]].astype(float)

We can change the order of columns again for a better picture

In [115]:
columns = df.columns
new_columns = (['Fighter A', 'Fighter B'] + 
               [col for col in columns if col not in ['Fighter A', 'Fighter B'] and not col.endswith((' A', ' B'))] + 
               [col for col in columns if col.endswith(' A') and col not in ['Fighter A']] +
               [col for col in columns if col.endswith(' B') and col not in ['Fighter B']])
print(new_columns, "\nNumber of cols:", len(new_columns))

['Fighter A', 'Fighter B', 'Result', 'Weightclass', 'Rounds', 'Venue', 'Date', 'Location', 'Billing', 'Event Name', 'Referee', 'Description', 'Method', 'Finish', 'Odds A', 'Title A', 'Weight A', 'Age A', 'Height A', 'Reach A', 'KO win A', 'KO loss A', 'Sub win A', 'Sub loss A', 'Dec win A', 'Dec loss A', 'Draws A', 'UFC win A', 'UFC loss A', 'UFC draws A', 'Nationality A', 'Nick A', 'Odds cat A', 'Odds B', 'Title B', 'Weight B', 'Age B', 'Height B', 'Reach B', 'KO win B', 'KO loss B', 'Sub win B', 'Sub loss B', 'Dec win B', 'Dec loss B', 'Draws B', 'UFC win B', 'UFC loss B', 'UFC draws B', 'Nationality B', 'Nick B', 'Odds cat B'] 
Number of cols: 52


In [116]:
df = df[new_columns]

In [117]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7294 entries, 0 to 7293
Data columns (total 52 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Fighter A      7294 non-null   object 
 1   Fighter B      7294 non-null   object 
 2   Result         7294 non-null   object 
 3   Weightclass    7294 non-null   float64
 4   Rounds         7294 non-null   int32  
 5   Venue          7294 non-null   object 
 6   Date           7294 non-null   object 
 7   Location       7294 non-null   object 
 8   Billing        7294 non-null   object 
 9   Event Name     7294 non-null   object 
 10  Referee        7192 non-null   object 
 11  Description    7265 non-null   object 
 12  Method         7294 non-null   object 
 13  Finish         7294 non-null   int32  
 14  Odds A         7294 non-null   int32  
 15  Title A        328 non-null    object 
 16  Weight A       7294 non-null   float64
 17  Age A          7294 non-null   int32  
 18  Height A     

We can see that after data wrangling we have 33 numerical columns and 19 categorical.

In [118]:
df.to_csv("ufc_data.csv", index=False)